In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Concatenate, GlobalAveragePooling2D, Dense, Flatten, Input
from tensorflow.keras.models import Model

large_conv_base = tf.keras.applications.MobileNetV3Large(weights='imagenet',
                                                         include_top=False,
                                                         input_shape=(224, 224, 3))

small_conv_base = tf.keras.applications.MobileNetV3Small(weights='imagenet',
                                                         include_top=False,
                                                         input_shape=(224, 224, 3))
print("MobileNetV3-Large Model Summary:")
large_conv_base.summary()
print("\nMobileNetV3-Small Model Summary:")
small_conv_base.summary()


In [ ]:
input_tensor = Input(shape=(224, 224, 3))

large_output = large_conv_base(input_tensor)
small_output = small_conv_base(input_tensor)

merged_output = Concatenate()([large_output, small_output])

flattened_output = Flatten()(merged_output)

dense_output = Dense(256, activation='relu')(flattened_output)

output = Dense(2, activation='softmax')(dense_output)

model = Model(inputs=input_tensor, outputs=output)

model.summary()


In [ ]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_1/train',# veri_seti_1/train | veri_seti_hard/Train
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    subset='training')

validation_generator = validation_datagen.flow_from_directory('/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_1/Validation',#veri_seti_1/Validation | veri_seti_hard/Validation
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='categorical',
                                                         subset='validation')

test_generator = test_datagen.flow_from_directory('/content/drive/My Drive/Image_Processing/Veri_Setleri/veri_seti_1/test', # veri_seti_1/test | veri_seti_hard/Test
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical')






In [ ]:
history = model.fit(train_generator,
                    epochs=15,
                    validation_data=validation_generator)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history.get('val_accuracy', [])
val_loss = history.history.get('val_loss', [])

epochs = range(1, len(train_acc) + 1)

plt.figure(figsize=(12, 5))

# Accuracy grafiği
plt.subplot(1, 2, 1)
plt.plot(epochs, train_acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss grafiği
plt.subplot(1, 2, 2)
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
